In [87]:
import torch 
import pandas as pd
import torch.utils
import torch.utils.data
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image 
import os
from pathlib import Path
from torchvision import transforms
from torch.optim import Adam # type: ignore
from torchvision import datasets
from torch.nn import functional as F

In [2]:
path = r'C:\Users\Anpanchiman\Downloads\Pistachio_Image_Dataset\Pistachio_Image_Dataset'
class_1_files = os.listdir(fr'{path}\Kirmizi_Pistachio')
Image.open(open(fr'{path}\Kirmizi_Pistachio\{class_1_files[1]}', 'rb'))

fr'{path}\{class_1_files[1]}' == fr'{path}\Kirmizi_Pistachio\kirmizi (1).jpg'

npIm = np.asarray(Image.open(fr'{path}\Kirmizi_Pistachio\kirmizi (1).jpg'))
npIm.shape

(600, 600, 3)

In [4]:
im = Image.open(fr'{path}\Kirmizi_Pistachio\kirmizi (1).jpg')

In [73]:
transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
])

In [16]:
def folder_train_test_split(path):
    files = os.listdir(path)
    return files[:int((7/10)*len(files))], list(np.setdiff1d(files, files[:int((7/10)*len(files))]))

In [25]:
train_class_1, test_class_1 = folder_train_test_split(fr'{path}\Kirmizi_Pistachio')
train_class_2, test_class_2 = folder_train_test_split(fr'{path}\Siirt_Pistachio')


In [60]:
print(f'{len(train_class_1)}, {len(train_class_2)}')
print(f'{len(test_class_1)}, {len(test_class_2)}')

862, 641
370, 275


In [79]:
class ImageFolderDataset(Dataset):
    def __init__(self, path, file_names, label, transform):
        self.path = path
        self.transform = transform
        self.label = label
        self.file_names = file_names
    
    def __len__(self):
        return len(self.file_names)
    
    def __getitem__(self, idx):
        im = Image.open(fr'{path}\{self.label}\{self.file_names[idx]}')
        if self.transform:
            im = self.transform(im)
        return im, self.label

In [82]:
def getData():
    training_set1 = ImageFolderDataset(fr'{path}\Kirmizi_Pistachio', train_class_1, 'Kirmizi_Pistachio', transform)
    training_set2 = ImageFolderDataset(fr'{path}\Siirt_Pistachio', train_class_2,'Siirt_Pistachio', transform)
    
    test_set1 = ImageFolderDataset(fr'{path}\Kirmizi_Pistachio', test_class_1, 'Kirmizi_Pistachio', transform)
    test_set2 = ImageFolderDataset(fr'{path}\Siirt_Pistachio', test_class_2, 'Siirt_Pistachio', transform)

    return ConcatDataset([training_set1, training_set2]), ConcatDataset([test_set1, test_set2])



In [83]:
train, test = getData()
print(f'{len(train)}, {len(test)}')

1503, 645


In [84]:
next(iter(train))

(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]),
 'Kirmizi_Pistachio')

In [94]:
class ImageClassifier(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        print('linear ok')
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)
        
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
            
clf = ImageClassifier()

linear ok


In [95]:
#Define loss function and optimiser
import torch.optim as opt

loss_fun = nn.CrossEntropyLoss()
optimiser = opt.SGD(clf.parameters(), lr=0.001, momentum=0.9)


In [97]:
running_loss = 0.0
for i, data in enumerate(train,0):
    inputs, labels = data
    optimiser.zero_grad()
    
    outputs = clf(inputs)
    print(type(outputs))
    loss = loss_fun(outputs, labels)
    loss.backward()
    optimiser.step()
    
    running_loss += loss.item()
    if i%2000 == 1999:
        print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
        running_loss = 0.0
    

RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x25 and 400x120)

In [ ]:
epochs = 3
for epoch in range(epochs):
    

In [22]:
clf = ImageClassifier()
opt = Adam(clf.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [21]:
training_set = datasets.FashionMNIST('./data', train=True, transform=transform, download=True)


<generator object Module.parameters at 0x000001BFC1D38C10>

In [24]:
training_loader = torch.utils.data.DataLoader(fr'{path}\Kirmizi_Pistachio\kirmizi (1).jpg', batch_size=4, shuffle=True)

In [3]:
training_set = Dataset(fr'{path}\Kirmizi_Pistachio')

TypeError: Dataset() takes no arguments